In [1]:
!pip install pymupdf pdfplumber pytesseract pillow langchain sentence-transformers regex langdetect qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 76.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00


In [3]:
import os
import fitz  # PyMuPDF
import pdfplumber
import pytesseract
from PIL import Image
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models
import regex as re
from langdetect import detect
import unicodedata
print("Done")

Done


In [32]:
from datetime import datetime

In [5]:
PDF_DIR = "/kaggle/input/my-pdfs"           # letakkan PDF disini
OUT_JSON_DIR = "/kaggle/working/pdf_json"
QDRANT_DIR = "/kaggle/working/qdrant_db" 

os.makedirs(OUT_JSON_DIR, exist_ok=True)
os.makedirs(QDRANT_DIR, exist_ok=True)

QDRANT_PATH = os.path.join(QDRANT_DIR, "qdrant_collection")
print("Cek")

Cek


In [6]:
!ls -R /kaggle/input


/kaggle/input:
my-pdfs

/kaggle/input/my-pdfs:
'Interpretable Machine Learning for COVID-19 An Empirical Study on Severity Prediction Task.pdf'
 Xie_Oriented_R-CNN_for_Object_Detection_ICCV_2021_paper.pdf


In [7]:
# ============================================================
# 4. Ekstraksi PDF (teks, tabel, gambar + OCR) - FIXED
# ============================================================
def extract_pdf(pdf_path, out_json_dir=OUT_JSON_DIR, image_ocr=True):
    doc = fitz.open(pdf_path)
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]

    data = {"pages": []}

    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")

        # OCR halaman jika teks terlalu sedikit
        if len(text.strip()) < 30:
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            text += "\n" + pytesseract.image_to_string(img)

        page_data = {
            "page_number": page_num + 1,
            "text": text.strip(),
            "tables": [],
            "images": []
        }

        # Ekstrak tabel dengan pdfplumber
        with pdfplumber.open(pdf_path) as plumber_pdf:
            plumber_page = plumber_pdf.pages[page_num]
            for table in plumber_page.find_tables():
                rows = table.extract()
                page_data["tables"].append({
                    "bbox": table.bbox,
                    "rows": rows
                })

        # Ekstrak gambar + OCR - FIXED
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            img_bytes = base_image["image"]
            img_ext = base_image["ext"]
            
            # FIX: Convert to bytes instead of bytearray
            img_bytes_obj = bytes(img_bytes)
            
            # Create in-memory file-like object
            from io import BytesIO
            img_buffer = BytesIO(img_bytes_obj)
            
            try:
                img_pil = Image.open(img_buffer)
                
                ocr_text = ""
                if image_ocr:
                    try:
                        ocr_text = pytesseract.image_to_string(img_pil)
                    except Exception as e:
                        print(f"OCR error: {e}")
                        ocr_text = ""
                
                page_data["images"].append({
                    "bbox": img[1],
                    "width": base_image.get("width"),
                    "height": base_image.get("height"),
                    "ext": img_ext,
                    "ocr_text": ocr_text.strip()
                })
                
            except Exception as e:
                print(f"Error processing image: {e}")
                # Still add image info even if processing failed
                page_data["images"].append({
                    "bbox": img[1],
                    "width": base_image.get("width"),
                    "height": base_image.get("height"),
                    "ext": img_ext,
                    "ocr_text": f"Error processing image: {str(e)}"
                })

        data["pages"].append(page_data)

    # Simpan ke JSON
    out_path = os.path.join(out_json_dir, f"{pdf_name}.json")
    with open(out_path, "w", encoding="utf-8") as fp:
        json.dump(data, fp, indent=2, ensure_ascii=False)

    print(f"✅ Ekstraksi selesai: {pdf_path} -> {out_path}")
    return out_path

print("Done Fixed")

Done Fixed


In [8]:
!pip install regex langdetect


In [10]:
import regex as re
from langdetect import detect
import unicodedata


In [11]:
def advanced_clean_text(text):
    # Unicode Normalization (NFKC)
    text = unicodedata.normalize('NFKC', text)

    # De-hyphenation
    text = re.sub(r'-\n', '', text)

    # Remove line breaks & multiple spaces
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    # Remove boilerplate: hapus baris pendek <5 karakter
    lines = text.split('\n')
    lines = [line for line in lines if len(line.strip()) > 5]
    text = "\n".join(lines)

    return text.strip()

print("Advanced Prepocess")

Advanced Prepocess


In [29]:
# ============================================================
#
# ============================================================
def json_to_documents(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    pdf_name = os.path.splitext(os.path.basename(json_path))[0]

    docs = []
    for page in data["pages"]:
        text_parts = [advanced_clean_text(page["text"])]

        # Tambahkan tabel sebagai markdown - FIXED
        for tbl in page["tables"]:
            # FIX: Handle None values in table rows
            cleaned_rows = []
            for r in tbl["rows"]:
                if r:  # hanya proses row yang tidak empty
                    # Convert semua cell ke string, handle None
                    cleaned_cells = [str(cell) if cell is not None else "" for cell in r]
                    cleaned_rows.append(" | ".join(cleaned_cells))
            
            if cleaned_rows:
                tbl_text = "\n".join(cleaned_rows)
                text_parts.append(f"TABLE:\n{tbl_text}")

        # Tambahkan OCR gambar
        for img in page["images"]:
            if img.get("ocr_text"):
                text_parts.append(f"IMAGE OCR:\n{img['ocr_text']}")

        combined = "\n".join(text_parts).strip()
        if combined:
            try:
                lang = detect(combined)
            except:
                lang = "unknown"

            docs.append(Document(
    page_content=combined,
    metadata={
        "document_id": f"{pdf_name}_page_{page['page_number']}",
        "source": os.path.basename(json_path),
        "file_type": "pdf",
        "created_at": datetime.now().isoformat(),
        "schema": "v1",
        "knowledge_type": "research_paper",  # atau lainnya
        "role_retention": "permanent",
        "page": page["page_number"],
        "lang": lang,
        "has_tables": len(page["tables"]) > 0,
        "has_images": len(page["images"]) > 0
    }
))
    return docs


print("============================================================")
print(" 6. Convert JSON ke LangChain Document & Chunking - FIXED .3")
print("============================================================")

 6. Convert JSON ke LangChain Document & Chunking - FIXED .3


In [21]:
print("============================================================")
print("7. Chunking + Embedding + Qdrant (Mengganti FAISS) - FIXED.3")
print("============================================================")

def process_documents(docs, qdrant_path=QDRANT_PATH, chunk_size=800, overlap=120):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    chunks = splitter.split_documents(docs)

    # Gunakan model yang lebih kecil dan cepat
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"  # Lebih cepat
    )

    # Buat Qdrant vector store
    vs = Qdrant.from_documents(
        chunks,
        embeddings,
        path=qdrant_path,  # Local path untuk Qdrant
        collection_name="pdf_documents",
        force_recreate=True,  # Buat collection baru setiap kali
        wait = True,
        timeout=60
    )

    print(f"✅ Qdrant vectorstore saved di {qdrant_path}")
    return vs

7. Chunking + Embedding + Qdrant (Mengganti FAISS) - FIXED.3


In [22]:
print("============================================================")
print("8. Review hasil ekstraksi (cek JSON & VectorDB) - FIXED.3")
print("============================================================")

def review_extraction(json_path):
    print(f"\n=== REVIEW EXTRACTION: {os.path.basename(json_path)} ===")
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    print(f"Jumlah halaman: {len(data['pages'])}")
    
    for page in data["pages"]:  # Review hanya 2 halaman pertama
        print(f"\n📄 Halaman {page['page_number']}")
        print(f"Teks ({len(page['text'])} chars): {page['text'][:200]}...")
        
        if page["tables"]:
            print(f"Jumlah tabel: {len(page['tables'])}")
            for i, tbl in enumerate(page["tables"][:1]):  # Review hanya tabel pertama
                print(f"  Tabel {i+1}:")
                for j, row in enumerate(tbl["rows"][:3]):  # Review hanya 3 baris pertama
                    # FIX: Handle None values in table cells
                    cleaned_row = [str(cell) if cell is not None else "" for cell in row]
                    print(f"    Baris {j+1}: {' | '.join(cleaned_row)}")
                if len(tbl["rows"]) > 3:
                    print("    ...")
        
        if page["images"]:
            print(f"Jumlah gambar: {len(page['images'])}")
            for i, img in enumerate(page["images"][:1]):  # Review hanya gambar pertama
                print(f"  Gambar {i+1}: {img.get('width', '?')}x{img.get('height', '?')}")
                if img.get("ocr_text"):
                    print(f"    OCR: {img['ocr_text'][:100]}...")

def check_json(out_json_dir=OUT_JSON_DIR):
    print("\n=== REVIEW JSON FILES ===")
    for f in os.listdir(out_json_dir):
        if f.endswith(".json"):
            path = os.path.join(out_json_dir, f)
            with open(path, "r", encoding="utf-8") as fp:
                data = json.load(fp)
            print(f"\n📄 File: {f}")
            print(f"Jumlah halaman: {len(data['pages'])}")
            for page in data["pages"][:1]:  # tampilkan contoh halaman 1
                print(f" Halaman {page['page_number']}")
                print("  Teks:", page["text"][:200], "...")
                if page["tables"]:
                    # FIX: Handle None values in table display
                    first_table = page["tables"][0]
                    if first_table["rows"]:
                        first_row = [str(cell) if cell is not None else "" for cell in first_table["rows"][0]]
                        print("  Tabel contoh:", " | ".join(first_row))
                if page["images"]:
                    print("  OCR Gambar contoh:", page["images"][0].get("ocr_text", "")[:100])

def check_vectorstore(qdrant_path=QDRANT_PATH):
    print("\n=== REVIEW QDRANT VECTORSTORE ===")
    try:
        # Gunakan SINGLE client instance
        client = QdrantClient(path=qdrant_path)
        
        # Beri waktu untuk sync
        import time
        time.sleep(1)
        
        # Check collection info
        collection_info = client.get_collection("pdf_documents")
        print(f"✅ Collection status: {collection_info.status}")
        
        # Count vectors
        count_result = client.count(
            collection_name="pdf_documents",
            exact=True
        )
        print(f"✅ Vectors count: {count_result.count}")
        
        client.close()
        
    except Exception as e:
        print(f"Error: {e}")

8. Review hasil ekstraksi (cek JSON & VectorDB) - FIXED.3


In [33]:
# Tambahkan di awal pipeline untuk reset
import shutil

# Hapus folder Qdrant lama jika ada
if os.path.exists(QDRANT_DIR):
    shutil.rmtree(QDRANT_DIR)
    print("🧹 Cleaned previous Qdrant storage")

os.makedirs(QDRANT_DIR, exist_ok=True)

🧹 Cleaned previous Qdrant storage


In [34]:
# ============================================================
# 9. Main Pipeline dengan Qdrant - FIXED
# ============================================================
print("============================================================")
print("9. Main Pipeline dengan Qdrant - FIXED")
print("============================================================")

# Clean previous storage
if os.path.exists(QDRANT_DIR):
    shutil.rmtree(QDRANT_DIR)
    print("🧹 Cleaned previous Qdrant storage")
os.makedirs(QDRANT_DIR, exist_ok=True)

pdf_files = [os.path.join(PDF_DIR, f) for f in os.listdir(PDF_DIR) if f.endswith(".pdf")]

for pdf in pdf_files:
    print(f"\n🔧 Processing: {os.path.basename(pdf)}")
    json_path = extract_pdf(pdf)
    review_extraction(json_path)
    docs = json_to_documents(json_path)
    process_documents(docs)  # ✅ Tidak ada concurrent access lagi

# Cek hasil ekstraksi
check_json()
check_vectorstore()  # ✅ Hanya satu client yang dibuka

print("Done")

9. Main Pipeline dengan Qdrant - FIXED
🧹 Cleaned previous Qdrant storage

🔧 Processing: Xie_Oriented_R-CNN_for_Object_Detection_ICCV_2021_paper.pdf
✅ Ekstraksi selesai: /kaggle/input/my-pdfs/Xie_Oriented_R-CNN_for_Object_Detection_ICCV_2021_paper.pdf -> /kaggle/working/pdf_json/Xie_Oriented_R-CNN_for_Object_Detection_ICCV_2021_paper.json

=== REVIEW EXTRACTION: Xie_Oriented_R-CNN_for_Object_Detection_ICCV_2021_paper.json ===
Jumlah halaman: 10

📄 Halaman 1
Teks (3508 chars): Oriented R-CNN for Object Detection
Xingxing Xie
Gong Cheng*
Jiabao Wang
Xiwen Yao
Junwei Han
School of Automation, Northwestern Ploytechnical University, Xi’an, China
{xiexing,jbwang}@mail.nwpu.edu.c...
Jumlah gambar: 1
  Gambar 1: 3940x920
    OCR: 54 anchors Oriented 2 anchors Horizontal Oriented 3 anchors Oriented
8 0°6 proposals proposals propo...

📄 Halaman 2
Teks (5917 chars): To push the envelope further: we investigate why the
efficiency of region proposal-based oriented detectors has
trailed thus far.
Our

In [25]:
def search_documents(query, k=3):
    """Search dalam vectorstore"""
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    
    vs = Qdrant(
        client=QdrantClient(path=QDRANT_PATH),
        collection_name="pdf_documents", 
        embeddings=embeddings
    )
    
    results = vs.similarity_search(query, k=k)
    
    print(f"🔍 Results for: '{query}'\n")
    for i, doc in enumerate(results):
        print(f"📄 Result {i+1} (Page {doc.metadata['page']}):")
        print(f"   {doc.page_content[:150]}...")
        print("-" * 80)

# Test search
search_documents("object detection")

🔍 Results for: 'object detection'

📄 Result 1 (Page 1):
   convolutional neural network (CNN) model can discriminates
COVID-19 from Non-COVID-19 using chest CT image [9].
It achieves interpretability through g...
--------------------------------------------------------------------------------
📄 Result 2 (Page 4):
   fine-tuned parameters, gradient boosting can result in better
performance than random forests. Still, it is tough for humans
to interpret a sequence o...
--------------------------------------------------------------------------------
📄 Result 3 (Page 1):
   various heterogeneous feature groups as input and outputs
an equidimensional embedding corresponding to each feature
group. The DeepFM [5] module comp...
--------------------------------------------------------------------------------
